##Usando o MatMiner e Python para visualização de dados
##Este notebook é uma continuação do anterior e agora vamos ver visualizar e ao mesmo tempo trabalhar com os dados obtidos.



In [ ]:
from platform import python_version
from matminer.data_retrieval.retrieve_MP import MPDataRetrieval
from matminer.data_retrieval.retrieve_Citrine import CitrineDataRetrieval
from matminer.datasets import load_dataset
from figrecipes import PlotlyFig #pacote para gráficos

#pacotes para ML
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

#computação cientifica e ta belas
import numpy as np
import pandas as pd

print(python_version())

In [ ]:
#importa dados do repositorio Citrine
ct = CitrineDataRetrieval("xT5QPhsWvXFYHn7P7XeqJQtt")
dc = ct.get_dataframe(criteria={'data_set_id': 154544, 'max_results':1000})
#limpeza de dados
dc = dc.drop(columns = ['references', 'Elongation in 2 in', 'Material Type-references', 'Reduction in Area', 'Elongation  (2 in)-units', 'Type-references', 'Yield Strength-conditions', 'ids', 'contacts', 'Reduction in Area-units', 'Impact Strength-units', 'preparation', 'Size', 'Elongation  (8 in)-references', 'Hardness', 'Hardness-references', 'Impact Strength-references'])
dc = dc.fillna(-1)
dc.describe()


In [ ]:
#inicio da visualização
plt = PlotlyFig(dc, x_title='Tensile Strength (Pa)', 
               y_title='Total', mode='notebook')
plt.histogram(dc['Tensile Strength'])
plt = PlotlyFig(dc, x_title='Tensile Strength (Pa)', 
               y_title='Elongation (lb/pol)', mode='notebook')
plt.xy(('Tensile Strength', 'Elongation'))
dc = dc.fillna(-1)
dc.head()

In [ ]:
#converte para números
dc['Tensile Strength'] = pd.to_numeric(dc['Tensile Strength'])
dc['Elongation'] = pd.to_numeric(dc['Elongation'])

#filtra os dados
dc = dc[dc['Tensile Strength'] > 1000]
dc = dc[dc['Elongation'] > 0]
dc.head()
dc.describe()


In [ ]:
plt = PlotlyFig(dc, x_title='Tensile Strength (Pa)', 
y_title='Elongation (lb/pol)', mode='notebook')
plt.xy(('Tensile Strength', 'Elongation'))

In [ ]:
#prepara os dados para ML
y = dc['Elongation'].values #valores a treinar
x = dc.drop('Elongation', axis=1) #descritores
print(y) 
print("Há {} Descritores possíveis:\n\n{}".format(x.shape[1], x.columns.values))

In [ ]:
x.head()

In [ ]:
limpar = ["names", "composition"]
x = x.drop(limpar, axis=1)
limpar = ["Tensile Strength-units", "Yield Strength-units", "Elongation-units"]
x = x.drop(limpar, axis=1)
limpar = ["Hardness-units", "Size-units"]
x = x.drop(limpar, axis=1)
limpar = ["Elongation in 2 in-units", "Material Type", "Form"]
x = x.drop(limpar, axis=1)
x.head()

In [ ]:
#cria um modelo simples
lr = LinearRegression()
lr.fit(x, y)
print('training R2 = ' + str(round(lr.score(x, y), 3)))
print('training RMSE = %.3f' % np.sqrt(mean_squared_error(y_true=y, y_pred=lr.predict(x))))